In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorica

In [ ]:
import pandas as pd
file_path = "/content/drive/MyDrive/Airline_Sentiment_Analysis.csv"

df = pd.read_csv(file_path)

In [ ]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
df['airline_sentiment'] = df['airline_sentiment'].replace('neutral', 1)
df['airline_sentiment'] = df['airline_sentiment'].replace('negative', 0)
df['airline_sentiment'] = df['airline_sentiment'].replace('positive', 2)


In [ ]:
X = df['text']
y = df['airline_sentiment']

In [ ]:
t = Tokenizer()
t.fit_on_texts(X)

In [ ]:
vocab_size = len(t.word_index) + 1


In [ ]:
sequences = t.texts_to_sequences(X)

In [ ]:
def max_tweet():
    for i in range(1, len(sequences)):
        max_length = len(sequences[0])
        if len(sequences[i]) > max_length:
            max_length = len(sequences[i])
    return max_length

In [ ]:
tweet_num = max_tweet()
tweet_num

30

In [ ]:
maxlen = tweet_num
padded_X = pad_sequences(sequences, padding='post', maxlen=maxlen)

In [ ]:
labels = to_categorical(np.asarray(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_X, labels, test_size = 0.2, random_state = 0)

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,100),
    tf.keras.layers.LSTM(256),
    tf.keras.layers.Dense(3,activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         1576900   
                                                                 
 lstm_1 (LSTM)               (None, 256)               365568    
                                                                 
 dense_1 (Dense)             (None, 3)                 771       
                                                                 
Total params: 1,943,239
Trainable params: 1,943,239
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['accuracy'])
history=model.fit(X_train,y_train,epochs=10,validation_split=0.2,batch_size=256)

Epoch 1/10
37/37 [==============================] - 23s 571ms/step - loss: 0.5613 - accuracy: 0.6143 - val_loss: 0.5216 - val_accuracy: 0.6219
Epoch 2/10
37/37 [==============================] - 20s 532ms/step - loss: 0.5012 - accuracy: 0.6390 - val_loss: 0.4969 - val_accuracy: 0.6287
Epoch 3/10
37/37 [==============================] - 19s 510ms/step - loss: 0.4462 - accuracy: 0.6735 - val_loss: 0.4424 - val_accuracy: 0.6995
Epoch 4/10
37/37 [==============================] - 21s 564ms/step - loss: 0.3992 - accuracy: 0.7149 - val_loss: 0.3902 - val_accuracy: 0.7213
Epoch 5/10
37/37 [==============================] - 20s 547ms/step - loss: 0.3592 - accuracy: 0.7490 - val_loss: 0.3834 - val_accuracy: 0.7256
Epoch 6/10
37/37 [==============================] - 18s 477ms/step - loss: 0.3376 - accuracy: 0.7696 - val_loss: 0.3749 - val_accuracy: 0.7618
Epoch 7/10
37/37 [==============================] - 21s 571ms/step - loss: 0.3056 - accuracy: 0.7997 - val_loss: 0.3716 - val_accuracy: 0.7721

In [ ]:
model.save("xyz.h5")

In [ ]:
airline = tf.keras.models.load_model('xyz.h5')

In [ ]:
results=airline.evaluate(X_test,y_test)
print(results)

92/92 [==============================] - 4s 34ms/step - loss: 0.4045 - accuracy: 0.7613
[0.4045352637767792, 0.7612704634666443]
